In [2]:
# load text books 
# extract vectors
# store the vectors in chromadb
# langchain chain for ollama call

In [8]:
import PyPDF2
import os
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# List of local PDF file paths for the three textbooks
pdf_files = ["TB/Washington.pdf"]

# Container to store all extracted text chunks
all_chunks = []

# Define the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)

# Process each PDF file
for file_path in pdf_files:
    extracted_text = extract_text_from_pdf(file_path)
    print(f"Extracted text from {file_path}: {extracted_text[:500]}")  # Print first 500 characters

    # Split the extracted text into chunks
    chunks = text_splitter.split_text(extracted_text)
    print(f"Number of chunks from {file_path}: {len(chunks)}")
    
    # Append the chunks to the all_chunks list
    all_chunks.extend(chunks)

# Create Document objects from the chunks
documents = [Document(page_content=chunk) for chunk in all_chunks]

# Specify the directory to store the Chroma vector database
persist_directory = "chroma_db"

# Ensure the directory exists
os.makedirs(persist_directory, exist_ok=True)

# Initialize the vector database using Ollama embeddings
vector_db = Chroma.from_documents(
    documents=documents,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    persist_directory=persist_directory,
    collection_name="local-rag"
)

# Output the number of documents in the vector database
print(f"Total number of documents in Chroma: {vector_db._collection.count()}")

Extracted text from TB/Washington.pdf: THE
 WASHINGTON MANUAL™ OF CRITICAL CARE
3rd Edition
Marin H. Kollef, MD
Virginia E. and Sam J. Golman
Chair in Respiratory Intensive Care Medicine
Professor of Medicine
Division of Pulmonary & Critical Care Medicine
Director, Critical Care Research
Director, Respiratory Care Services
Washington University School of Medicine
Barnes-Jewish Hospital
St. Louis, Missouri
Warren Isakow, MD
Associate Professor of Medicine
Division of Pulmonary and Critical Care Medicine
Director, Medical Intensive Car
Number of chunks from TB/Washington.pdf: 252


OllamaEmbeddings: 100%|██████████| 252/252 [27:37<00:00,  6.58s/it]


Total number of documents in Chroma: 252


In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
local_model = "llama2"
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(search_kwargs={"k": 2}),
    llm=llm,
    prompt=QUERY_PROMPT
)

# # RAG prompt
# template = """Answer the question based ONLY on the following context:
# {context}
# Question: {question}
# Make sure the answer is SHORT, CRISP and generated FAST. Try to keep it in one line.
# """

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What is the document about?")